<a href="https://colab.research.google.com/github/RJRuhan/notebook/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Thesis/codes
%ls

/content/drive/MyDrive/Thesis/codes
extract_feature.py  IO_helpers.py      plot.py        __pycache__/  verisense_step_count.py
features.csv        modified_dataset/  preprocess.py  trainData.py


In [2]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn import preprocessing
from sklearn import svm
import matplotlib.pylab as plt
from plot import *
from extract_feature import *
from trainData import *

%matplotlib

Using matplotlib backend: <object object at 0x7b0735b27660>


In [4]:
def hyperparameters_opt_RBF(X_train, y_train, X_test, y_test,
                            param_range_C, param_range_gamma):

    accuracy_matrix = np.zeros((param_range_C.shape[0],param_range_gamma.shape[0]))
    for i,C in enumerate(param_range_C):
        for j,gamma in enumerate(param_range_gamma):
            clf_r_l = svm.SVC(kernel='poly', C=C, gamma=gamma, degree=3)
            clf_r_l.fit(X_train, y_train)
            accuracy_matrix[i,j] = f1_score(y_test ,clf_r_l.predict(X_test), average='macro')
    C_ind, gamma_ind = np.unravel_index(np.argmax(accuracy_matrix), accuracy_matrix.shape)
    # print(accuracy_matrix)

    return param_range_C[C_ind], param_range_gamma[gamma_ind]

In [5]:
X_train_sets, Y_train_sets, X_test_sets, Y_test_sets, X_validation_set, Y_validation_set, X_train, Y_train = load_dataset_feature_gender('features.csv', n_folds=7)
print(len(X_train_sets))
print(X_train_sets[0].shape)
print(Y_train_sets[0].shape)
print(X_test_sets[0].shape)
print(Y_test_sets[0].shape)
print(X_validation_set.shape)
print(Y_validation_set.shape)
print(X_train.shape)
print(Y_train.shape)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1]
7
(266, 409)
(266,)
(46, 409)
(46,)
(134, 409)
(134,)
(312, 409)
(312,)


In [6]:

num_set = len(X_train_sets)

for i in range(num_set):
  # C and gamma test range
  param_range = np.array([0.01, 0.1, 1, 10, 100, 1000])
  # Optimizing C and gamma parameters to achieve the best f1 score
  C, gamma = hyperparameters_opt_RBF(X_train_sets[i], Y_train_sets[i], X_test_sets[i], Y_test_sets[i], param_range, param_range)

  print("Hyperparameters optimization results:")
  print("C: {}".format(C))
  print("gamma: {}".format(gamma))
  print("\n")


Hyperparameters optimization results:
C: 0.01
gamma: 0.01


Hyperparameters optimization results:
C: 0.01
gamma: 0.01


Hyperparameters optimization results:
C: 0.01
gamma: 0.01


Hyperparameters optimization results:
C: 0.01
gamma: 0.01


Hyperparameters optimization results:
C: 0.01
gamma: 0.01


Hyperparameters optimization results:
C: 0.01
gamma: 0.01


Hyperparameters optimization results:
C: 0.01
gamma: 0.01




In [7]:
C = 0.01
gamma = 0.01
label_names = ['male','female']
clf_r_l = svm.SVC(kernel='poly', C=C, gamma=gamma, degree=3)
for i in range(num_set):
  # Training the model with the optimized hyperparameters
  clf_r_l.fit(X_train_sets[i], Y_train_sets[i])
  y_pred_tr = clf_r_l.predict(X_train_sets[i])
  y_pred_te = clf_r_l.predict(X_test_sets[i])

  print('Training set report')
  print(classification_report(Y_train_sets[i], y_pred_tr, target_names=label_names))
  print('Test set report')
  print(classification_report(Y_test_sets[i], y_pred_te, target_names=label_names))

clf_r_l.fit(X_train, Y_train)
y_pred_cov = clf_r_l.predict(X_validation_set)
print('Validation set report')
print(classification_report(Y_validation_set, y_pred_cov, target_names=label_names))

Training set report
              precision    recall  f1-score   support

        male       1.00      1.00      1.00       133
      female       1.00      1.00      1.00       133

    accuracy                           1.00       266
   macro avg       1.00      1.00      1.00       266
weighted avg       1.00      1.00      1.00       266

Test set report
              precision    recall  f1-score   support

        male       0.56      0.61      0.58        23
      female       0.57      0.52      0.55        23

    accuracy                           0.57        46
   macro avg       0.57      0.57      0.56        46
weighted avg       0.57      0.57      0.56        46

Training set report
              precision    recall  f1-score   support

        male       1.00      1.00      1.00       133
      female       1.00      1.00      1.00       133

    accuracy                           1.00       266
   macro avg       1.00      1.00      1.00       266
weighted avg      